In [2]:
import os
from googleapiclient.discovery import build as build_api_client

In [3]:
assert 'YOUTUBE_API_KEY' in os.environ
yt = build_api_client('youtube', 'v3', developerKey=os.environ['YOUTUBE_API_KEY'])

AssertionError: 

In [3]:
def load_categories(video_ids):
    request = yt.videos().list(
        part='snippet',
        id=','.join(video_ids)
    )
    response = request.execute()
    return {item['id']: item['snippet']['categoryId'] for item in response['items']}

In [4]:
import numpy as np
import tqdm.notebook
from loaders import load_features

tqdm.notebook.tqdm()
features = load_features()

In [10]:
unknown_ids = features[features['category_id'].isnull()]['video_id']

print(len(unknown_ids))

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

categories_dict = {}

videos_per_query = 20
for chunk in tqdm.tqdm(chunker(unknown_ids, videos_per_query), total=len(unknown_ids)/videos_per_query):
    video_ids = list(chunk)
    response = load_categories(video_ids)
    categories_dict.update(response)
    for vid, category in response.items():
        features.loc[features['video_id'] == vid, 'category_id'] = float(category)

  0%|          | 0/32.35 [00:00<?, ?it/s]

647


33it [00:03, 10.45it/s]                           


In [11]:
features['category_id'].isnull().sum()

647

In [14]:
features.to_csv('../youtube_data/features_with_categories.csv')